In [1]:
from pyspark.sql import SparkSession
import pyspark.pandas as ps
import datetime as dt
from src.emulator import generate_transactions_table
from src.emulator import generate_customer_profiles_table
from src.emulator import generate_terminal_profiles_table
from src.emulator import get_list_terminals_within_radius
from src.emulator import generate_dataset, add_frauds

In [2]:
spark = (
    SparkSession.builder.master("yarn")
        .appName("OTUS")
        # .config("spark.yarn.queue", "root")
        .getOrCreate()
)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
2022-12-12 09:31:51,124 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-12-12 09:31:54,122 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


Py4JError: org.apache.spark.api.python.PythonUtils.getPythonAuthSocketTimeout does not exist in the JVM

In [ ]:
(customer_profiles_table_df, terminal_profiles_table_df, transactions_df)=\
    generate_dataset(n_customers = 5000, 
                     n_terminals = 10000, 
                     nb_days=250, 
                     start_date=dt.date(year=2021, month=1, day=1).isoformat(), 
                     r=5)

transactions_sdf=spark.createDataFrame(transactions_df) 
transactions_sdf.write.mode("append").parquet("hdfs:///card-fraud-detection/transactions.parquet")

terminal_profiles_table_sdf=spark.createDataFrame(terminal_profiles_table_df) 
terminal_profiles_table_sdf.write.mode("append").parquet("hdfs:///card-fraud-detection/customer_profiles_table.parquet")

terminal_profiles_table_sdf=spark.createDataFrame(terminal_profiles_table_df) 
terminal_profiles_table_sdf.write.mode("append").parquet("hdfs:///card-fraud-detection/terminal_profiles_table.parquet")    

In [ ]:
transactions_sdf.count()

In [ ]:
spark.stop()